In [2]:
import numpy as np  
import math
import pandas as pd 
import os, csv, sys
import glob
import subprocess
from tqdm.notebook import tqdm
from dataclasses import dataclass
from scipy.interpolate import InterpolatedUnivariateSpline
from datetime import datetime, timezone, timedelta

folder_path = os.path.abspath('gnss_analysis')
sys.path.append(folder_path)
from gnssutils import EphemerisManager,WLS,Troposphere,Ionosphere,ElevationAzimuthAngle

In [3]:
#define constance
LIGHTSPEED = 299_792_458

Data

In [4]:
path = '2020-06-25-00-34-us-ca-mtv-sb-101\device_gnss_2020_6.csv'
gnss = pd.read_csv(path, index_col=False)
filtered_gnss = gnss[gnss['ReceivedSvTimeNanos'] > 1e10]
selected_col = ['ConstellationType','Svid','utcTimeMillis','ReceivedSvTimeNanos','TimeOffsetNanos','IsrbMeters',
                'RawPseudorangeMeters','Cn0DbHz','MultipathIndicator','RawPseudorangeUncertaintyMeters']
measurement = filtered_gnss[selected_col]
truth = gnss[gnss['ReceivedSvTimeNanos'] > 1e10]



In [5]:
# Lọc các vệ tinh thuộc GPS (ConstellationType == 1)
measurement.loc[measurement['ConstellationType'] == 1, 'Constellation'] = 'G'
truth.loc[truth['ConstellationType'] == 1,'Constellation'] = 'G'
# Chỉ lấy các vệ tinh bắt GPS
measurement = measurement.loc[measurement['Constellation'] == 'G']
truth = truth.loc[truth['Constellation'] == 'G']
# Đảm bảo cột 'Svid' là chuỗi ký tự để có thể sử dụng hàm str.len()
measurement['Svid'] = measurement['Svid'].astype(str)
# Thêm số 0 vào trước 'Svid' nếu chiều dài của nó là 1
measurement.loc[measurement['Svid'].str.len() == 1, 'Svid'] = '0' + measurement['Svid']
# Tạo cột mới 'SvName' bằng cách nối 'Constellation' và 'Svid'
measurement['SvName'] = measurement['Constellation'] + measurement['Svid']
truth



C:\Users\Admin\AppData\Local\Temp\ipykernel_13688\1035592269.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  measurement.loc[measurement['ConstellationType'] == 1, 'Constellation'] = 'G'


,MessageType,utcTimeMillis,TimeNanos,LeapSecond,TimeUncertaintyNanos,FullBiasNanos,BiasNanos,BiasUncertaintyNanos,DriftNanosPerSecond,DriftUncertaintyNanosPerSecond,...,SvVelocityZEcefMetersPerSecond,SvClockBiasMeters,SvClockDriftMetersPerSecond,IsrbMeters,IonosphericDelayMeters,TroposphericDelayMeters,WlsPositionXEcefMeters,WlsPositionYEcefMeters,WlsPositionZEcefMeters,Constellation
0,Raw,1593045252440,1047929361000000,18,NaN,-1276032541079953176,0.038029,19.234471,-23.264117,11.553386,...,2921.007727,-114240.991171,-0.002747,0.000000,2.990605,3.330763,-2.692783e+06,-4.297236e+06,3.855231e+06,G
1,Raw,1593045252440,1047929361000000,18,NaN,-1276032541079953176,0.038029,19.234471,-23.264117,11.553386,...,727.894656,6344.271476,0.000551,0.000000,4.233219,9.598289,-2.692783e+06,-4.297236e+06,3.855231e+06,G
2,Raw,1593045252440,1047929361000000,18,NaN,-1276032541079953176,0.038029,19.234471,-23.264117,11.553386,...,1639.029553,-66543.908717,-0.000458,0.000000,3.189919,4.111997,-2.692783e+06,-4.297236e+06,3.855231e+06,G
3,Raw,1593045252440,1047929361000000,18,NaN,-1276032541079953176,0.038029,19.234471,-23.264117,11.553386,...,-2495.388750,-52349.705491,-0.001107,0.000000,4.307212,5.539522,-2.692783e+06,-4.297236e+06,3.855231e+06,G
4,Raw,1593045252440,1047929361000000,18,NaN,-1276032541079953176,0.038029,19.234471,-23.264117,11.553386,...,-1274.485806,68762.686062,0.003014,0.000000,2.381497,2.830938,-2.692783e+06,-4.297236e+06,3.855231e+06,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46601,Raw,1593046551440,1049228361000000,18,NaN,-1276032541079925211,0.382517,21.912852,5.968073,10.719493,...,2640.901448,91727.335176,0.002122,0.000000,6.767570,12.101899,-2.706386e+06,-4.275965e+06,3.869231e+06,G
46617,Raw,1593046551440,1049228361000000,18,NaN,-1276032541079925211,0.382517,21.912852,5.968073,10.719493,...,1035.128340,-11610.693714,-0.000516,-2346.758209,9.285365,31.248352,-2.706386e+06,-4.275965e+06,3.869231e+06,G
46618,Raw,1593046551440,1049228361000000,18,NaN,-1276032541079925211,0.382517,21.912852,5.968073,10.719493,...,2638.837065,-114245.110320,-0.002746,-2346.758209,4.670052,2.992559,-2.706386e+06,-4.275965e+06,3.869231e+06,G
46619,Raw,1593046551440,1049228361000000,18,NaN,-1276032541079925211,0.382517,21.912852,5.968073,10.719493,...,-3109.899075,69416.213397,0.002549,-2346.758209,10.859158,9.168051,-2.706386e+06,-4.275965e+06,3.869231e+06,G


In [6]:
#chuyển giá trị thời gian về dạng date time
measurement['UnixTime'] = pd.to_datetime(measurement['utcTimeMillis'],unit='ms', utc=True)
#Chia measurement thành các epoch, mỗi epoch gồm các vệ tinh có cùng thời gian thu
measurement['Epoch'] = 0
measurement.loc[measurement['UnixTime'] - measurement['UnixTime'].shift() > timedelta(milliseconds=200), 'Epoch'] = 1
measurement['Epoch'] = measurement['Epoch'].cumsum()

In [7]:
# Khởi tạo biến để lưu trữ kết quả
filtered_measurement = pd.DataFrame()

# Duyệt qua từng epoch
for epoch in measurement['Epoch'].unique():
    one_epoch = measurement.loc[measurement['Epoch'] == epoch].drop_duplicates(subset='SvName')
    # Thêm kết quả vào DataFrame kết quả
    filtered_measurement = pd.concat([filtered_measurement, one_epoch])

# Reset index của DataFrame mới
filtered_measurement.reset_index(drop=True, inplace=True)
filtered_measurement = filtered_measurement.groupby('Epoch').apply(lambda x: x.sort_values(by='Svid')).reset_index(drop=True)
measurement = filtered_measurement

C:\Users\Admin\AppData\Local\Temp\ipykernel_13688\3100079312.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_measurement = filtered_measurement.groupby('Epoch').apply(lambda x: x.sort_values(by='Svid')).reset_index(drop=True)


In [8]:
#lấy dữ liệu ephemeris
ephemeris_data_directory = 'output'
manager = EphemerisManager(ephemeris_data_directory)

measurement['UnixTime'] = pd.to_datetime(measurement['utcTimeMillis'],unit='ms', utc=True)
timestamps = pd.to_datetime(measurement['UnixTime'].unique(),unit='s')
ephemeris_data = pd.DataFrame()

for timestamp in timestamps:
    # Get satellite names for the current timestamp
    sats = measurement.loc[measurement['UnixTime'] == timestamp, 'SvName'].tolist()
    # Retrieve ephemeris data
    ephemeris = manager.get_ephemeris(timestamp, sats)
    # Append the retrieved ephemeris data to the list
    ephemeris_data = pd.concat([ephemeris_data,ephemeris])

ephemeris_data.reset_index(drop=True, inplace=True)

Tính toán svPosition và svVelocity

In [9]:
def calculate_satellite_position_in_GPS(ephemeris, transmit_time):
    mu = 3.986005e14
    OmegaDot_e = 7.2921151467e-5
    F = -4.442807633e-10
    sv_position = pd.DataFrame()
    sv_position['sv']= ephemeris.index
    sv_position.set_index('sv', inplace=True)
    sv_position['t_k'] = transmit_time - ephemeris['t_oe']
    A = ephemeris['sqrtA'].pow(2)
    n_0 = np.sqrt(mu / A.pow(3))
    n = n_0 + ephemeris['deltaN']
    M_k = ephemeris['M_0'] + n * sv_position['t_k']
    E_k = M_k
    err = pd.Series(data=[1]*len(sv_position.index))
    i = 0
    
    # Eccentric anomaly
    while err.abs().min() > 1e-8 and i < 10:
        new_vals = M_k + ephemeris['e']*np.sin(E_k)
        err = new_vals - E_k
        E_k = new_vals
        i += 1
        
    sinE_k = np.sin(E_k)
    cosE_k = np.cos(E_k)
    delT_r = F * ephemeris['e'].pow(ephemeris['sqrtA']) * sinE_k
    delT_oc = transmit_time - ephemeris['t_oc']
    sv_position['delT_sv'] = ephemeris['SVclockBias'] + ephemeris['SVclockDrift'] * delT_oc + ephemeris['SVclockDriftRate'] * delT_oc.pow(2)

    v_k = np.arctan2(np.sqrt(1-ephemeris['e'].pow(2))*sinE_k,(cosE_k - ephemeris['e']))

    Phi_k = v_k + ephemeris['omega']

    sin2Phi_k = np.sin(2*Phi_k)
    cos2Phi_k = np.cos(2*Phi_k)

    du_k = ephemeris['C_us']*sin2Phi_k + ephemeris['C_uc']*cos2Phi_k
    dr_k = ephemeris['C_rs']*sin2Phi_k + ephemeris['C_rc']*cos2Phi_k
    di_k = ephemeris['C_is']*sin2Phi_k + ephemeris['C_ic']*cos2Phi_k

    u_k = Phi_k + du_k

    r_k = A*(1 - ephemeris['e']*np.cos(E_k)) + dr_k

    i_k = ephemeris['i_0'] + di_k + ephemeris['IDOT']*sv_position['t_k']

    x_k_prime = r_k*np.cos(u_k)
    y_k_prime = r_k*np.sin(u_k)

    Omega_k = ephemeris['Omega_0'] + (ephemeris['OmegaDot'] - OmegaDot_e)*sv_position['t_k'] - OmegaDot_e*ephemeris['t_oe']

    sv_position['x_k'] = x_k_prime*np.cos(Omega_k) - y_k_prime*np.cos(i_k)*np.sin(Omega_k)
    sv_position['y_k'] = x_k_prime*np.sin(Omega_k) + y_k_prime*np.cos(i_k)*np.cos(Omega_k)
    sv_position['z_k'] = y_k_prime*np.sin(i_k)
    sv_position['SvClockBiasMeters'] = ephemeris['SVclockBias'] * LIGHTSPEED
    sv_position['SvClockDriftMetersPerSecond'] = ephemeris['SVclockDrift'] * LIGHTSPEED
    #calculate velocuty
    Edot_k = n_0/(1-ephemeris['e']*np.cos(E_k))
    
    vdot_k = Edot_k * np.sqrt(1 - ephemeris['e']**2)/(1 - ephemeris['e']*np.cos(E_k))
    
    d_idotk = ephemeris['IDOT'] + 2 * vdot_k * (ephemeris['C_is'] * cos2Phi_k - ephemeris['C_ic']*sin2Phi_k)
    
    udot_k = vdot_k + 2 * vdot_k * (ephemeris['C_us']*cos2Phi_k - ephemeris['C_uc']*sin2Phi_k)
    
    rdot_k = ephemeris['e'] * A * Edot_k * np.sin(E_k) + 2 * vdot_k * (ephemeris['C_rs'] * cos2Phi_k - ephemeris['C_rc'] * sin2Phi_k)
    
    Omegadot_k = ephemeris['OmegaDot'] - OmegaDot_e
    
    x_plus_dot_k = rdot_k * np.cos(u_k) - r_k * udot_k*np.sin(u_k)
    y_plus_dot_k = rdot_k * np.sin(u_k) + r_k * udot_k*np.cos(u_k)
    
    sv_position['v_x'] = -x_k_prime * Omegadot_k * np.sin(Omega_k) + x_plus_dot_k * np.cos(Omega_k) - y_plus_dot_k * np.sin(Omega_k)*np.cos(i_k) - y_k_prime * (Omegadot_k * np.cos(Omega_k) * np.cos(i_k) - d_idotk * np.sin(Omega_k)*np.sin(i_k))
    
    sv_position['v_y'] = x_k_prime * Omegadot_k * np.cos(Omega_k) + x_plus_dot_k * np.sin(Omega_k) + y_plus_dot_k * np.cos(Omega_k)*np.cos(i_k) - y_k_prime * (Omegadot_k * np.sin(Omega_k) * np.cos(i_k) + d_idotk * np.cos(Omega_k)*np.sin(i_k))
    
    sv_position['v_z'] = y_plus_dot_k * np.sin(i_k) + y_k_prime * d_idotk * np.cos(i_k)
    
    
    return sv_position

In [10]:
# Run the function and check out the results:
measurement['ReceivedSvTimeNanos'] = pd.to_numeric(measurement['ReceivedSvTimeNanos'])
measurement['TimeOffsetNanos'] = pd.to_numeric(measurement['TimeOffsetNanos'])
sv_position = calculate_satellite_position_in_GPS(ephemeris_data,  1e-9*(measurement['ReceivedSvTimeNanos'] + measurement['TimeOffsetNanos']))
measurement['tTxSeconds'] = 1e-9*(measurement['ReceivedSvTimeNanos'] + measurement['TimeOffsetNanos'])
measurement['SvPositionXEcefMeters'] = sv_position['x_k']
measurement['SvPositionYEcefMeters'] = sv_position['y_k']
measurement['SvPositionZEcefMeters'] = sv_position['z_k']
measurement['SvVelocityXEcefMetersPerSecond'] = sv_position['v_x']
measurement['SvVelocityYEcefMetersPerSecond'] = sv_position['v_y']
measurement['SvVelocityZEcefMetersPerSecond'] = sv_position['v_z']
measurement['SvClockBiasMeters'] = sv_position['SvClockBiasMeters']
measurement['SvClockDriftMetersPerSecond'] = sv_position['SvClockDriftMetersPerSecond']



In [11]:
# measurement[['SvPositionXEcefMeters','SvPositionYEcefMeters','SvPositionZEcefMeters','tTxSeconds']].head(20)
measurement[['ReceivedSvTimeNanos']]

,ReceivedSvTimeNanos
0,347670368542152
1,347670360573483
2,347670367737558
3,347670363511034
4,347670371934131
...,...
14849,348969370005975
14850,348969360286476
14851,348969365828788
14852,348969360716360


In [12]:
def my_ecef2lla(pos):
  # x, y and z are scalars or vectors in meters
  x = pos[0]
  y = pos[1]
  z = pos[2]
  a=6378137
  a_sq=a**2
  e = 8.181919084261345e-2
  e_sq = 6.69437999014e-3

  f = 1/298.257223563
  b = a*(1-f)

  # calculations:
  r = np.sqrt(x**2 + y**2)
  ep_sq  = (a**2-b**2)/b**2
  ee = (a**2-b**2)
  f = (54*b**2)*(z**2)
  g = r**2 + (1 - e_sq)*(z**2) - e_sq*ee*2
  c = (e_sq**2)*f*r**2/(g**3)
  s = (1 + c + np.sqrt(c**2 + 2*c))**(1/3.)
  p = f/(3.*(g**2)*(s + (1./s) + 1)**2)
  q = np.sqrt(1 + 2*p*e_sq**2)
  r_0 = -(p*e_sq*r)/(1+q) + np.sqrt(0.5*(a**2)*(1+(1./q)) - p*(z**2)*(1-e_sq)/(q*(1+q)) - 0.5*p*(r**2))
  u = np.sqrt((r - e_sq*r_0)**2 + z**2)
  v = np.sqrt((r - e_sq*r_0)**2 + (1 - e_sq)*z**2)
  z_0 = (b**2)*z/(a*v)
  h = u*(1 - b**2/(a*v))
  phi = np.arctan((z + ep_sq*z_0)/r)
  lambd = np.arctan2(y, x)

  return phi*180/np.pi, lambd*180/np.pi, h

Tính toàn evelation và azimuth

In [13]:
# Đường dẫn đến file RINEX navigation của bạn
get_alpha_beta = manager.get_filepaths(timestamp)
path = get_alpha_beta['nasa_daily_gps']['filepath']
file_with_extension = os.path.basename(path)
file_name, extension = os.path.splitext(file_with_extension)
file_name = "output/nasa/" + file_name
alpha, beta = manager.extract_ionospheric_params(file_name)

In [14]:
wls = WLS()
angle = ElevationAzimuthAngle()
calculate_tropo = Troposphere()
calculate_iono = Ionosphere()
ecef_list = []
df = pd.DataFrame(measurement)
for epoch in measurement['Epoch'].unique():
    measurement['IsrbMeters'].loc[len(measurement['IsrbMeters'])] = 0
    one_epoch = measurement.loc[(measurement['Epoch'] == epoch)] 
    if len(one_epoch.index) > 4:
        x = wls.WLS_onePosition_rawPseudo(one_epoch)
        estimated_lla = my_ecef2lla(x)
        for index in one_epoch.index:
            result_list = measurement.loc[index, ['SvPositionXEcefMeters', 'SvPositionYEcefMeters', 'SvPositionZEcefMeters']]
            satellite_ecef = (result_list)
            elevation, azimuth = angle.CalculateAngle(satellite_ecef,estimated_lla)

            # Thêm các giá trị mới vào DataFrame
            measurement.at[index, 'SvElevationDegrees'] = elevation
            measurement.at[index, 'SvAzimuthDegrees'] = azimuth
    
        

C:\Users\Admin\AppData\Local\Temp\ipykernel_13688\3751700785.py:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  measurement['IsrbMeters'].loc[len(measurement['IsrbMeters'])] = 0
C:\Users\Admin\AppData\Local\Temp\ipykernel_13688\3751700785.p

position lsq status = 2



C:\Users\Admin\AppData\Local\Temp\ipykernel_13688\3751700785.py:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  measurement['IsrbMeters'].loc[len(measurement['IsrbMeters'])] = 0
C:\Users\Admin\AppData\Local\Temp\ipykernel_13688\3751700785.p

position lsq status = 2



C:\Users\Admin\AppData\Local\Temp\ipykernel_13688\3751700785.py:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  measurement['IsrbMeters'].loc[len(measurement['IsrbMeters'])] = 0
C:\Users\Admin\AppData\Local\Temp\ipykernel_13688\3751700785.p

...

C:\Users\Admin\AppData\Local\Temp\ipykernel_13688\3751700785.py:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  measurement['IsrbMeters'].loc[len(measurement['IsrbMeters'])] = 0
C:\Users\Admin\AppData\Local\Temp\ipykernel_13688\3751700785.p

position lsq status = 2



C:\Users\Admin\AppData\Local\Temp\ipykernel_13688\3751700785.py:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  measurement['IsrbMeters'].loc[len(measurement['IsrbMeters'])] = 0
C:\Users\Admin\AppData\Local\Temp\ipykernel_13688\3751700785.p

position lsq status = 2



C:\Users\Admin\AppData\Local\Temp\ipykernel_13688\3751700785.py:8: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  measurement['IsrbMeters'].loc[len(measurement['IsrbMeters'])] = 0
C:\Users\Admin\AppData\Local\Temp\ipykernel_13688\3751700785.p

In [15]:
measurement[['SvElevationDegrees','SvAzimuthDegrees']].head(50)
# measurement.to_csv('gnss_sat.csv')

,SvElevationDegrees,SvAzimuthDegrees
0,48.562346,242.995305
1,14.893346,42.686116
2,37.351635,64.248032
3,26.708073,282.268680
4,61.942392,49.833297
5,77.850553,297.959847
6,59.171792,323.265544
7,21.724948,244.481500
8,24.717031,316.832679
9,26.211019,146.561420


In [16]:
for epoch in measurement['Epoch'].unique():
    measurement['IsrbMeters'].loc[len(measurement['IsrbMeters'])] = 0
    one_epoch = measurement.loc[(measurement['Epoch'] == epoch)] 
    if len(one_epoch.index) > 4:
        x = wls.WLS_onePosition_rawPseudo(one_epoch)
        estimated_lla = my_ecef2lla(x)
        for index in one_epoch.index:
            result_list = measurement.loc[index, ['SvPositionXEcefMeters', 'SvPositionYEcefMeters', 'SvPositionZEcefMeters']]
            satellite_ecef = (result_list)
            measurement.at[index,'TroposphericDelayMeters'] = calculate_tropo.calculateTropoCorrectionMeters(
                math.radians(measurement['SvElevationDegrees'].iloc[index]),
                math.radians(estimated_lla[0]),
                estimated_lla[2],
                measurement['utcTimeMillis'].values)

            measurement.at[index,'IonosphericDelayMeters'] = calculate_iono.klobuchar_ionospheric_delay(
                measurement['utcTimeMillis'].values,
                estimated_lla[0],
                estimated_lla[1],
                measurement['SvAzimuthDegrees'].iloc[index],
                measurement['SvElevationDegrees'].iloc[index],
                alpha,
                beta)
            
            

C:\Users\Admin\AppData\Local\Temp\ipykernel_13688\727363032.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  measurement['IsrbMeters'].loc[len(measurement['IsrbMeters'])] = 0
C:\Users\Admin\AppData\Local\Temp\ipykernel_13688\727363032.py:

position lsq status = 2



C:\Users\Admin\AppData\Local\Temp\ipykernel_13688\727363032.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  measurement['IsrbMeters'].loc[len(measurement['IsrbMeters'])] = 0
C:\Users\Admin\AppData\Local\Temp\ipykernel_13688\727363032.py:

position lsq status = 2



C:\Users\Admin\AppData\Local\Temp\ipykernel_13688\727363032.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  measurement['IsrbMeters'].loc[len(measurement['IsrbMeters'])] = 0
C:\Users\Admin\AppData\Local\Temp\ipykernel_13688\727363032.py:

...

C:\Users\Admin\AppData\Local\Temp\ipykernel_13688\727363032.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  measurement['IsrbMeters'].loc[len(measurement['IsrbMeters'])] = 0
C:\Users\Admin\AppData\Local\Temp\ipykernel_13688\727363032.py:

position lsq status = 2



C:\Users\Admin\AppData\Local\Temp\ipykernel_13688\727363032.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  measurement['IsrbMeters'].loc[len(measurement['IsrbMeters'])] = 0
C:\Users\Admin\AppData\Local\Temp\ipykernel_13688\727363032.py:

position lsq status = 2



C:\Users\Admin\AppData\Local\Temp\ipykernel_13688\727363032.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  measurement['IsrbMeters'].loc[len(measurement['IsrbMeters'])] = 0
C:\Users\Admin\AppData\Local\Temp\ipykernel_13688\727363032.py:

In [20]:
mea = pd.DataFrame()
t = pd.DataFrame()
mea['TroposphericDelayMeters1'] = measurement['TroposphericDelayMeters']
mea['utcTimeMillis'] = measurement['utcTimeMillis']

t['TroposphericDelayMeters'] = truth['TroposphericDelayMeters']
t['utcTimeMillis'] = truth['utcTimeMillis']

merged_df = pd.merge(mea,t,how = 'inner', on='utcTimeMillis', suffixes=('_1', '_2'))

merged_df = merged_df.dropna()
merged_df

,TroposphericDelayMeters1,TroposphericDelayMeters,dif
0,3.355810,3.330763,0.025048
1,9.788211,9.598289,0.189922
2,4.146614,4.111997,0.034617
3,5.597509,5.539522,0.057987
4,2.850813,2.830938,0.019876
...,...,...,...
14835,6.119637,2.552370,3.567267
14836,3.847152,2.844391,1.002761
14837,3.310447,7.363264,4.052818
14838,3.529627,6.364366,2.834739


In [ ]:
# mea = pd.DataFrame()
# t = pd.DataFrame()
# mea['SvElevationDegrees'] = measurement['SvElevationDegrees']
# mea['utcTimeMillis'] = measurement['utcTimeMillis']
# t['SvElevationDegrees'] = truth['SvElevationDegrees']
# t['utcTimeMillis'] = truth['utcTimeMillis']
# merged_df = pd.merge(mea,t,how = 'inner', on='utcTimeMillis', suffixes=('_1', '_2'))
# merged_df = merged_df.dropna()
# merged_df.head(50)


,SvElevationDegrees_1,utcTimeMillis,SvElevationDegrees_2
0,48.562346,1593045252440,48.562397
1,48.562346,1593045252440,14.893310
2,48.562346,1593045252440,37.351636
3,48.562346,1593045252440,26.708022
4,48.562346,1593045252440,61.942349
5,48.562346,1593045252440,77.850424
6,48.562346,1593045252440,59.171852
7,48.562346,1593045252440,21.724997
8,48.562346,1593045252440,26.211205
9,48.562346,1593045252440,3.135079
